# Example Process

## Get Data 
(In actual application, this would be done with database queries)

In [1]:
from pathlib import Path
import json
import pandas as pd

In [2]:
data_path = Path(r"C:\Users\lhi30\Haein\2023\YBIGTA\2023-2\Conference\HDMEDI\Data\seventh.csv")
data= pd.read_csv(data_path, index_col = 0)

## Split Sections using split_sections()

In [3]:
import hd_medi_data

In [4]:
data["Processed"] = data["주의사항"].apply(hd_medi_data.split_sections)
data

,품목기준코드,의약품명,주의사항,주성분 영문,component,Processed
0,202001274,글로틴듀오정2.5/500mg,NaN,Metformin Hydrochloride,1,{}
1,202001275,글로틴듀오정2.5/850mg,NaN,Linagliptin,1,{}
2,202001275,글로틴듀오정2.5/850mg,NaN,Metformin Hydrochloride,1,{}
3,202001276,글로틴듀오정2.5/1000mg,NaN,Linagliptin,1,{}
4,202001276,글로틴듀오정2.5/1000mg,NaN,Metformin Hydrochloride,1,{}
...,...,...,...,...,...,...
2467,201508553,슈가메트서방정5/1000밀리그램,1. 경고\n메트포르민염산염\n1) 드물게 심한 유산산증(lactic acidosi...,[M223014]메트포르민염산염/[M244179]메트포르민염산염/[M263350]에...,3,"{'1. 경고': ['메트포르민염산염', '1) 드물게 심한 유산산증(lactic ..."
2468,201700836,디유비정,"1. 다음과 같은 사람은 이 약을 복용하지 말 것.\n1) 급성 복부질환(충수염, ...",[M050687]비사코딜/[M227778]디메티콘/[M223167]우르소데옥시콜산,3,{'1. 다음과 같은 사람은 이 약을 복용하지 말 것.': ['1) 급성 복부질환(...
2469,201705576,스토마제정,1. 다음과 같은 사람은 이 약을 복용하지 말 것.\n1) 만7세 이하의 어린이\n...,[M268886]우담즙건조엑스/[M100297]셀룰라제/[M040678]판크레아틴/...,3,{'1. 다음과 같은 사람은 이 약을 복용하지 말 것.': ['1) 만7세 이하의 ...
2470,201706690,듀오그린정,"1. 다음과 같은 사람은 이 약을 복용하지 말 것.\n1) 급성 복부질환(충수염, ...",[M237598]Tinnevelly 센나열매가루/[M050687]비사코딜/[M081...,3,{'1. 다음과 같은 사람은 이 약을 복용하지 말 것.': ['1) 급성 복부질환(...


## For one medicine, construct tagged dataframe using make_df()

In [5]:
# import keywords
json_path = Path(r"C:\Users\lhi30\Haein\2023\YBIGTA\2023-2\Conference\HDMEDI\Share\Ybigta_HDMedi\Software\keywords_ybigta.json")
with open(json_path, 'r', encoding='utf-8') as json_file:
        default_keywords = json.load(json_file)

In [30]:
default_keywords

{'forbid': ['투여하지 말 것',
  '경고',
  '금기사항',
  '복용하지 말 것',
  '복용하지 마십시오',
  '복용(사용)하지 말 것',
  '투여하지 마십시오',
  '하지 말 것',
  '즉각 중지',
  '중지'],
 'warning': ['신중히 투여할 것',
  '주의사항',
  '주의',
  '신중하게 투여할 것',
  '신중투여 할 것',
  '와 상의할 것',
  '주의할 사항'],
 'child': ['소아',
  '아이',
  '[0-9]+세 이하',
  '영유아',
  '신생아',
  '유아',
  '어린이',
  '미취학 아동',
  '학령기 아동',
  '첫해',
  '개월령',
  '생후 [0-9]+개월',
  '생후 [0-9]+일',
  '초등학생',
  '유치원생',
  '청소년',
  '학령 전 아동',
  '청소년기',
  '사춘기',
  '성장기',
  '영아'],
 'pregnant': ['임신중의', '수유중의', '임부', '임신하고 있을 가능성', '수유부', '모유', '여성', '가임기'],
 'elderly': ['고령자', '[0-9]+세 이상'],
 'male': ['남성', '남자'],
 'female': ['여성', '여자']}

## Additional Tagging 

In [6]:
def demo_additional_tagging(receive_user_input: bool, example_df: pd.DataFrame, past_med_list:list = [], interest_dis_list:list =[]):
    if receive_user_input:
        past_medicine = input("What medicine have you had before? If you want to quit entering, input \"q\": ")
        while past_medicine != "q":
            past_med_list.append(past_medicine)
            past_medicine = input("What medicine have you had before? If you want to quit entering, input \"q\": ")
        disease = input("What diseases are you interested in?If you want to quit entering, input \"q\": ")
        while disease != "q":
            interest_dis_list.append(disease)
            disease = input("What diseases are you interested in?If you want to quit entering, input \"q\": ")
        meds_dict, disease_dict = hd_medi_data.process_user_input(past_med_list, interest_dis_list)
        result_df = hd_medi_data.additional_tagging(example_df, meds_dict, disease_dict)
    else:
        meds_dict, disease_dict = hd_medi_data.process_user_input(past_med_list, interest_dis_list)
        result_df = hd_medi_data.additional_tagging(example_df, meds_dict, disease_dict)
    return result_df


## Reorganization of Data
- Disease Interest
    - Gather all of those with the same disease of interest 
    - Order each content based on importance
- Past Medicine
    - Gather all of those with the same medicine tag
    - Order each content based on importance
- Topics
    - Figure out if user is child, pregnant, or elderly
    - Order based on importance
- Display original
    - Just get the original

In [7]:
def get_disease_results(disease_masked, disease_tag_int):
    disease_int_tag = {value:key for key, value in disease_tag_int.items()}
    df_exploded = disease_masked.explode('Disease Interest')
    df_sorted = df_exploded.sort_values(by=["Section Importance"])
    df_grouped = df_sorted.groupby(["Disease Interest"])
    resulting_dict = {}
    for name, grouped_rows in df_grouped:
        heading = f"{disease_int_tag[name[0]]}을/를 겪는 사람들을 위한 주의사항"
        group_by_section = grouped_rows[['Section', 'Content']].groupby(["Section"])
        text_list = []
        for section, content in group_by_section:
            section_heading = f"{section[0]}"
            section_content = "\n".join(content['Content'])
            text = section_heading + ":\n" + section_content
            text_list.append(text)
        resulting_dict[heading] = "\n".join(text_list)
    return resulting_dict

def get_medicine_results(past_medicine_masked, medicine_tag_int):
    medicine_int_tag = {value:key for key, value in medicine_tag_int.items()}
    df_exploded = past_medicine_masked.explode('Past Medicine')
    df_sorted = df_exploded.sort_values(by=["Section Importance"])
    df_grouped = df_sorted.groupby(["Past Medicine"])
    resulting_dict = {}
    for name, grouped_rows in df_grouped:
        heading = f"{medicine_int_tag[name[0]]}을/를 사용하는 사람들을 위한 주의사항"
        group_by_section = grouped_rows[['Section', 'Content']].groupby(["Section"])
        text_list = []
        for section, content in group_by_section:
            section_heading = f"{section[0]}"
            section_content = "\n".join(content['Content'])
            text = section_heading + ":\n" + section_content
            text_list.append(text)
        resulting_dict[heading] = "\n".join(text_list)
    return resulting_dict

def order_base_data(topic_masked, user_input, topic_to_tag, topic_translation):
    tag_to_topic = {value:key for key, value in topic_to_tag.items()}
    df_exploded = topic_masked.explode('Topics')
    df_sorted = df_exploded.sort_values(by=['Section Importance'])
    resulting_dict = {}
    for tag, user_yes in user_input.items():
        if user_yes:
            heading = f"{topic_translation[tag_to_topic[tag]]} 위한 주의사항"
            grouped_rows = df_sorted[df_sorted['Topics'] == tag]
            group_by_section = grouped_rows[['Section', 'Content']].groupby(["Section"])
            text_list = []
            for section, content in group_by_section:
                section_heading = f"{section[0]}"
                section_content = "\n".join(content['Content'])
                text = section_heading + ":\n" + section_content
                text_list.append(text)
            if text_list:
                resulting_dict[heading] = "\n".join(text_list)
    return resulting_dict
            

In [8]:
topic_translation = {
    'child': "유아 & 청소년을",
    'pregnant': "임산부를",
    'elderly': "고령자를",
    'male': "남성을",
    'female': "여성을"
}
def overall_ordering(result_df, topic_to_tag, tags_int, original_data, user_input ,topic_translation = topic_translation):
    topic_mask = result_df["Topics"].apply(lambda x: len(x)>0)
    topic_masked = result_df[topic_mask]
    disease_tag_int = tags_int['disease']
    medicine_tag_int = tags_int['medicine']

    overall_result = {}
    if disease_tag_int:
        disease_interest_mask = result_df["Disease Interest"].apply(lambda x: len(x) > 0)
        disease_masked = result_df[disease_interest_mask]
        overall_result.update(get_disease_results(disease_masked, disease_tag_int))
    if medicine_tag_int:
        past_medicine_mask = result_df["Past Medicine"].apply(lambda x: len(x)>0)
        past_medicine_masked = result_df[past_medicine_mask]
        overall_result.update(get_medicine_results(past_medicine_masked, medicine_tag_int))
    overall_result.update(order_base_data(topic_masked, user_input, topic_to_tag, topic_translation))
    overall_result["original"] = original_data

    return overall_result
    

In [35]:
user_name = "father"
number = 27
display(data.iloc[number, :])
example_data = data["Processed"][number]
original_data = data["주의사항"][number]
interest_disease_list = ['심부전증','당뇨']
past_medicine_list = ['이부프로펜', '아스피린']
user_input =  {
    0: False,
    1: False,
    2: False,
    3: True,
    4: False
}
display(original_data)

품목기준코드                                               202001373
의약품명                                                모아펜소프트연질캡슐
주의사항         1. 경고\n1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해...
주성분 영문                                               Ibuprofen
component                                                    1
Processed    {'1. 경고': ['1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나...
Name: 27, dtype: object

'1. 경고\n1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열진통제를 복용해야 할 경우 반드시 의사 또는 약사와 상의해야 한다. 이러한 사람이 이 약을 복용하면 위장출혈이 유발될 수 있다.\n2) 심혈관계 위험: 조절되지 않는 고혈압, 울혈심부전증(NYHA II-III), 확립된 허혈성 심장질환, 말초동맥질환, 뇌혈관질환을 가진 환자들은 신중히 고려하여 이부프로펜을 사용하여야하며 고용량 이부프로펜(1일 2400mg) 사용을 피해야 한다. 또한 심혈관계 위험 요소(예. 고혈압, 고지혈증, 당뇨병, 흡연)를 가지고 있는 환자가 고용량 이부프로펜(1일 2400mg)이 필요한 경우 장기간 치료를 시작하기 전에 신중히 고려해야한다.\n임상연구 결과 고용량(1일 2400mg) 이부프로펜 사용이 동맥 혈전 증상(심근경색증 또는 뇌졸중)에 대한 위험성을 다소 증가시킬 수 있다고 나타났다. 종합적으로 역학연구 결과 저용량 이부프로펜(예. 1일 1200 mg 이하)과 동맥 혈전 증상의 위험성 증가간의 연관성은 증명되지 않았다.\n2. 다음과 같은 사람은 이 약을 복용하지 말 것.\n1) 이 약 및 이 약의 구성성분에 대한 과민반응 및 그 병력이 있는 사람\n2) 이 약 및 이 약의 구성성분, 다른 해열진통제, 감기약 복용 시 천식을 일으킨 적이 있는 사람\n3) 만3개월 미만의 영아(갓난아기)\n3. 이 약을 복용하는 동안 다음의 약을 복용하지 말 것.\n다른 해열진통제, 감기약, 진정제\n4. 다음과 같은 사람은 이 약을 복용하기 전에 의사, 치과의사, 약사와 상의 할 것.\n1) 수두 또는 인플루엔자에 감염되어 있거나 또는 의심되는 영아(갓난아기) 및 만 15세 미만의 어린이(구역이나 구토를 수반하는 행동의 변화가 있다면, 드물지만 심각한 질병인 레이증후군의 초기 증상일 수 있으므로 의사와 상의할 것.)\n2) 만 3 개월 이상 만 1세 미만의 영아(갓난아기)는 의사의 진료를 받는 것을 우선으로 하며, 부득이한 경우를 제외하고는 복용시키지 않는 것이

In [36]:
def final_process(example_data, interest_disease_list, past_medicine_list, original_data, user_input):
    example_df, topic_to_tag = hd_medi_data.make_df(example_data, default_keywords)
    result_df, tags_int = demo_additional_tagging(False, example_df, past_medicine_list, interest_disease_list)
    result_json = overall_ordering(result_df, topic_to_tag, tags_int, original_data =original_data, user_input=user_input)
    return result_json

result = final_process(example_data, interest_disease_list, past_medicine_list, original_data = original_data, user_input=user_input)
for heading, content in result.items():
    print(f"- {heading}")
    print(content)

json_path = Path(r"C:\Users\lhi30\Haein\2023\YBIGTA\2023-2\Conference\HDMEDI\Data\character_json")
with open(json_path / f"{user_name}.json", 'w', encoding='utf-8') as json_file:
        json.dump(result, json_file)

- 심부전증을/를 겪는 사람들을 위한 주의사항
경고:
심혈관계 위험: 조절되지 않는 고혈압, 울혈심부전증(NYHA II-III), 확립된 허혈성 심장질환, 말초동맥질환, 뇌혈관질환을 가진 환자들은 신중히 고려하여 이부프로펜을 사용하여야하며 고용량 이부프로펜(1일 2400mg) 사용을 피해야 한다. 또한 심혈관계 위험 요소(예. 고혈압, 고지혈증, 당뇨병, 흡연)를 가지고 있는 환자가 고용량 이부프로펜(1일 2400mg)이 필요한 경우 장기간 치료를 시작하기 전에 신중히 고려해야한다.
- 당뇨을/를 겪는 사람들을 위한 주의사항
경고:
심혈관계 위험: 조절되지 않는 고혈압, 울혈심부전증(NYHA II-III), 확립된 허혈성 심장질환, 말초동맥질환, 뇌혈관질환을 가진 환자들은 신중히 고려하여 이부프로펜을 사용하여야하며 고용량 이부프로펜(1일 2400mg) 사용을 피해야 한다. 또한 심혈관계 위험 요소(예. 고혈압, 고지혈증, 당뇨병, 흡연)를 가지고 있는 환자가 고용량 이부프로펜(1일 2400mg)이 필요한 경우 장기간 치료를 시작하기 전에 신중히 고려해야한다.
다음과 같은 사람은 이 약을 복용하기 전에 의사, 치과의사, 약사와 상의 할 것.:
간장질환, 신장(콩팥)질환, 갑상선질환, 당뇨병, 고혈압 등이 있는 사람, 몸이 약한 사람 또는 고열이 있는 사람
의사 또는 치과의사의 치료를 받고 있는 사람(당뇨약, 통풍약, 관절염약, 항응고제, 스테로이드제 등 다른 약물을 투여 받고 있는 사람)
- 이부프로펜을/를 사용하는 사람들을 위한 주의사항
경고:
심혈관계 위험: 조절되지 않는 고혈압, 울혈심부전증(NYHA II-III), 확립된 허혈성 심장질환, 말초동맥질환, 뇌혈관질환을 가진 환자들은 신중히 고려하여 이부프로펜을 사용하여야하며 고용량 이부프로펜(1일 2400mg) 사용을 피해야 한다. 또한 심혈관계 위험 요소(예. 고혈압, 고지혈증, 당뇨병, 흡연)를 가지고 있는 환자가 고용량 이부프로펜(1일 2400mg)이 필요한 경우 장기간 치료를 시작하기 전에 신중히 고